# Brusselator model

The dynamics of the oscillating reaction discovered by Belousov and Zhabotinsky, 
can be modeled through the so-called Brusselator model depending on two parameters:

$$
\left\{\begin{aligned}
\frac{dy_1}{dt} & = a - (b+1) y_1 + y_1^2y_2\\
\frac{dy_2}{dt} & = b y_1 - y_1^2y_2
\end{aligned}\right.
$$

In [1]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import Label

from ipywidgets import interact, FloatSlider

from mylib.model import brusselator_model
import mylib.continuation as continuation

output_notebook(hide_banner=True)

In [24]:


bm = brusselator_model(a=1, b=3.)
fcn = bm.fcn
jac_eq = bm.jac_eq

tini = 0. 
tend = 80.

yini = (1.5,0)

tol = 1.e-10
sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
eig_vals_eq, _= np.linalg.eig(jac_eq())

fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
plt_y1 = fig_sol.line(sol.t, sol.y[0], legend="y1", line_width=2)    
plt_y2 = fig_sol.line(sol.t, sol.y[1], legend="y2", line_width=2, color="Green")
fig_sol.legend.location = "top_left"

fig_pha = figure(plot_height=300, plot_width=475, title="Phase plan")
plt_pha = fig_pha.line(sol.y[0], sol.y[1], line_width=2,color='Blue')

a=sol.y[0]
b=sol.y[1]
for i in range(len(a)):
    if(a[i]>3.5 and a[i]>a[i+1]):
        break;
print(a[i],b[i])

for j in range(i, len(a)):
    a[j]=a[i]
    b[j]=0
a[-1]=yini[0]
b[-1]=yini[1]

fig_pha.line(a, b, line_width=2,color='Red')  
fig_pha.x(a[i-1],b[i-1],line_width=10 ,color='Black')
fig_pha.x(a[i],0,line_width=10 ,color='Green')
fig_pha.x(yini[0],yini[1],line_width=10 ,color='Blue')



fig_eig = figure(x_range=(-2, 2), y_range=(-2, 2), plot_height=300, plot_width=475, 
                 title="Eigen values at equilibrium points")
fig_eig.line(x=0, y=np.linspace(-2.,2,10), color="black")
fig_eig.line(x=np.linspace(-2.,2,10), y=0, color="black")
plt_eig = fig_eig.circle(np.real(eig_vals_eq), np.imag(eig_vals_eq), line_width=2, size=10)

show(column(fig_sol, row(fig_pha, fig_eig)), notebook_handle=True)

def update(a):
    bm = brusselator_model(a, b=3.)
    fcn = bm.fcn
    jac_eq = bm.jac_eq
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
    eig_vals_eq, _= np.linalg.eig(jac_eq())
    plt_y1.data_source.data = dict(x=sol.t, y=sol.y[0])
    plt_y2.data_source.data = dict(x=sol.t, y=sol.y[1])
    plt_pha.data_source.data = dict(x=sol.y[0], y=sol.y[1])
    plt_eig.data_source.data = dict(x=np.real(eig_vals_eq), y=np.imag(eig_vals_eq))
    push_notebook()

interact(update, a=FloatSlider(min=np.sqrt(2)-1.0, max=np.sqrt(2)+1.0, value=1, step=0.01, 
                               continuous_update=False))



3.7684654086838294 0.989358551274426


interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='a', max=2.414213562373095, …

<function __main__.update(a)>

* $-x^2*y^2+(x^3+2*x^2+3*x)*y+4*x^2-12*x-1$

In [27]:
0.989*3.77**2-4*3.77+1

-0.023441899999999904

## Natural parameter continuation

In [3]:
def show_natural_continuation():
    
    a_ini = 0.9
    a_end = 8.0
    na = 201
    fig_branch_y1 = figure(plot_height=450, plot_width=950)
    fig_branch_y1.xaxis.axis_label = "a"
    fig_branch_y1.yaxis.axis_label = "y1"    
    fig_branch_y2 = figure(plot_height=450, plot_width=950)
    fig_branch_y2.xaxis.axis_label = "a"
    fig_branch_y2.yaxis.axis_label = "y2"   
    
    yeq_ini = (a_ini, 3/a_ini)
    b=3
    list_b=[3,10,20,30]
    for b in list_b:
        yeq = continuation.natural_continuation_order_0(a_ini, a_end, na, yeq_ini, brusselator_model,b)
        #print(len(yeq[0]))
        #yeq = continuation.natural_continuation_order_1(a_ini, a_end, na, yeq_ini, brusselator_model,b)
        #print(len(yeq[0]))
        a = np.linspace(a_ini, a_end, na)
        a_stable   = a[np.where(a > np.sqrt(b-1))]
        yeq_stable   = yeq[:, np.where(a > np.sqrt(b-1))[0]]
        a_unstable   = a[np.where(a < np.sqrt(b-1))]
        yeq_unstable = yeq[:, np.where(a < np.sqrt(b-1))[0]]
        yhopf = 0.5*(yeq_stable[:,0] + yeq_unstable[:,-1])
        fig_branch_y1.x(a_stable, yeq_stable[0], color="green", legend="stable branch for y1")
        fig_branch_y1.x(a_unstable, yeq_unstable[0], color="red", legend="unstable branch for y1")
        fig_branch_y1.x(np.sqrt(b-1), yhopf[0], size=15, line_width=3, legend="Hopf bifurcation")
        fig_branch_y1.legend.location = "top_left"



        fig_branch_y2.x(a_stable, yeq_stable[1], color="green", legend="stable branch for y2")
        fig_branch_y2.x(a_unstable, yeq_unstable[1], color="red", legend="unstable branch for y2")
        fig_branch_y2.x(np.sqrt(b-1), yhopf[1], size=15, line_width=3, legend="Hopf bifurcation")

    x=np.linspace(1,8,num=100)
    fig_branch_y2.x(x,(x**2+1)/x)
    show(column(fig_branch_y1, fig_branch_y2))

show_natural_continuation()

* 第一个图像是linear的，所以根本不用hopf，可以直接解，手算就行了
* newton解，用不同的方法，不同阶的
* following the branch which is potentially instable, continuation algorithms


In [18]:
np.linspace(1,20,num=100)

array([ 1.        ,  1.19191919,  1.38383838,  1.57575758,  1.76767677,
        1.95959596,  2.15151515,  2.34343434,  2.53535354,  2.72727273,
        2.91919192,  3.11111111,  3.3030303 ,  3.49494949,  3.68686869,
        3.87878788,  4.07070707,  4.26262626,  4.45454545,  4.64646465,
        4.83838384,  5.03030303,  5.22222222,  5.41414141,  5.60606061,
        5.7979798 ,  5.98989899,  6.18181818,  6.37373737,  6.56565657,
        6.75757576,  6.94949495,  7.14141414,  7.33333333,  7.52525253,
        7.71717172,  7.90909091,  8.1010101 ,  8.29292929,  8.48484848,
        8.67676768,  8.86868687,  9.06060606,  9.25252525,  9.44444444,
        9.63636364,  9.82828283, 10.02020202, 10.21212121, 10.4040404 ,
       10.5959596 , 10.78787879, 10.97979798, 11.17171717, 11.36363636,
       11.55555556, 11.74747475, 11.93939394, 12.13131313, 12.32323232,
       12.51515152, 12.70707071, 12.8989899 , 13.09090909, 13.28282828,
       13.47474747, 13.66666667, 13.85858586, 14.05050505, 14.24